In [17]:
# Access Neo4J credentials
NEO4J_URI=""
NEO4J_USERNAME=""
NEO4J_PASSWORD=""

In [18]:
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

In [6]:
# Movie Dataset
movie_query = """
    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row

    MERGE (m:Movie {id:row.movieId})
    SET m.released = date(row.released),
        m.title = row.title,
        m.imdbRating = toFloat(row.imdbRating)
    FOREACH (director IN split(row.director, '|') |
        MERGE (p:Person {name:trim(director)})
        MERGE (p)-[:DIRECTED]->(m)
    )
    FOREACH (actor IN split(row.actors, '|') |
        MERGE (p:Person {name:trim(actor)})
        MERGE (p)-[:ACTED_IN]->(m)
    )
    FOREACH (genre IN split(row.genres, '|') |
        MERGE (g:Genre {name:trim(genre)})
        MERGE (m)-[:IN_GENRE]->(g)
    )
"""

In [7]:
movie_query

"\n    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row\n\n    MERGE (m:Movie {id:row.movieId})\n    SET m.released = date(row.released),\n        m.title = row.title,\n        m.imdbRating = toFloat(row.imdbRating)\n    FOREACH (director IN split(row.director, '|') |\n        MERGE (p:Person {name:trim(director)})\n        MERGE (p)-[:DIRECTED]->(m)\n    )\n    FOREACH (actor IN split(row.actors, '|') |\n        MERGE (p:Person {name:trim(actor)})\n        MERGE (p)-[:ACTED_IN]->(m)\n    )\n    FOREACH (genre IN split(row.genres, '|') |\n        MERGE (g:Genre {name:trim(genre)})\n        MERGE (m)-[:IN_GENRE]->(g)\n    )\n\n"

In [8]:
graph.query(movie_query)

[]

In [9]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {POB: STRING, DOB: INTEGER, name: STRING}
Company {name: STRING}
Developer {POB: STRING, name: STRING, YOB: INTEGER}
Country {name: STRING}
Person {name: STRING, Born: INTEGER}
Movie {name: STRING, Released: INTEGER, id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [11]:
# Load env data
import os
from dotenv import load_dotenv
load_dotenv()

# Groq API Key
groq_api_key = os.getenv("GROQ_API_KEY")

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x72bea7132180>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x72bea7132a20>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x72bef06695e0>, cypher_generation_chain=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x72bea7132180>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x72bea7132a20>, model_name='gemma2-9b-it', model_kwargs

In [20]:
response = chain.invoke({"query": "Give me list of Tom Hunks movies?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN m.title 

Full Context:
[{'m.title': None}, {'m.title': 'Toy Story'}, {'m.title': 'Apollo 13'}]

> Finished chain.


{'query': 'Give me list of Tom Hunks movies?',
 'result': 'Toy Story, Apollo 13 \n'}

In [23]:
chain.invoke({ "query": "Give me highest IMDB rating movie"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)
RETURN m.imdbRating ORDER BY m.imdbRating DESC LIMIT 1;
Full Context:
[{'m.imdbRating': None}]

> Finished chain.


{'query': 'Give me highest IMDB rating movie',
 'result': "I don't know the answer. \n"}

In [24]:
chain.invoke({ "query": "Who acted in Forest Gump movie?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name: "Forest Gump"})<-[:ACTED_IN]-(p:Person) RETURN p.name  

Full Context:
[{'p.name': 'Tom Hanks'}]

> Finished chain.


{'query': 'Who acted in Forest Gump movie?',
 'result': 'Tom Hanks  acted in Forest Gump movie. \n'}

In [26]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name: 'Top Gun'})-[:ACTED_IN]->(p:Person) RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?', 'result': "I don't know the answer. \n"}